In [1]:
# Roll Number: 22CD91F01
# Name of the student: Nazeer Haider
# Project Code: SRHC-AS
# Project Title: Sales Grouping by Representatives using Single Linkage Agglomerative (Bottom-Up) Clustering Technique

# Import the libraries
import numpy as np
import pandas as pd

In [2]:
# Load the sales data from CSV file
sales_data = pd.read_csv("sales.csv")
sales_data = sales_data.sample(1000) # Run on full dataset it takes long time, so I have used just 1000 samples, you can run with full dataset by comment this line
sales_data.head()

,Country,Record,Deal ID,Sales Rep,Sales,Year of Transaction Date
136044,Spain,1,ES-2013-2239615,Ralph Ritter,530.868,2013
147440,United Kingdom,1,MX-2013-101525,Frank Atkinson,295.452,2013
230105,Sudan,1,CA-2011-124464,Shirley Schmidt,39.480,2011
170724,France,1,ID-2012-29424,Elizabeth Moffitt,1041.336,2012
260142,United Kingdom,1,MX-2011-106390,Patrick O'Donnell,195.440,2011


In [3]:
# Drop the unnecessary columns from the data
sales_data = sales_data.drop("Record", axis=1)

In [4]:
# In dataset "Deat ID" have a mixture of "Category-Year-ID". So I have split this column on the basis of "-".
# And you have taken Category and ID, date is already in dataset.
# new data frame with split value columns
new = sales_data["Deal ID"].str.split("-", n = 2, expand = True)
 
# making separate first name column from new data frame
sales_data["Category"]= new[0]
 
# making separate last name column from new data frame
sales_data["ID"]= new[2]
sales_data['ID'] = sales_data['ID'].astype(str).astype(int)
 
# Dropping old Name columns
sales_data.drop(columns =["Deal ID"], inplace = True)
 
# df display
sales_data

,Country,Sales Rep,Sales,Year of Transaction Date,Category,ID
136044,Spain,Ralph Ritter,530.868,2013,ES,2239615
147440,United Kingdom,Frank Atkinson,295.452,2013,MX,101525
230105,Sudan,Shirley Schmidt,39.480,2011,CA,124464
170724,France,Elizabeth Moffitt,1041.336,2012,ID,29424
260142,United Kingdom,Patrick O'Donnell,195.440,2011,MX,106390
...,...,...,...,...,...,...
212252,Sweden,Shahid Collister,14.910,2012,CA,113740
93775,Central African Republic,Brendan Sweed,30.600,2013,ES,1026422
209653,Colombia,Maribeth Dona,105.420,2012,CA,134782
80944,Kenya,Ann Chong,12.768,2014,CA,101637


In [5]:
# Encoding the categorical parameters
encoded_sales_data = pd.get_dummies(sales_data, columns = ['Country', 'Category', 'Sales Rep'])
encoded_sales_data

,Sales,Year of Transaction Date,ID,Country_Afghanistan,Country_Algeria,Country_Angola,Country_Argentina,Country_Australia,Country_Austria,Country_Azerbaijan,...,Sales Rep_Victoria Brennan,Sales Rep_Victoria Pisteka,Sales Rep_Victoria Wilson,Sales Rep_Vivek Gonzalez,Sales Rep_Vivek Grady,Sales Rep_Vivek Sundaresam,Sales Rep_William Brown,Sales Rep_Xylona Preis,Sales Rep_Yoseph Carroll,Sales Rep_Zuschuss Carroll
136044,530.868,2013,2239615,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
147440,295.452,2013,101525,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230105,39.480,2011,124464,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
170724,1041.336,2012,29424,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
260142,195.440,2011,106390,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212252,14.910,2012,113740,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93775,30.600,2013,1026422,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209653,105.420,2012,134782,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
80944,12.768,2014,101637,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Convert the dataset into numpy array for ease of use
encoded_sales_data = encoded_sales_data.to_numpy()

In [7]:
# Define a function to compute the cosine similarity between two vectors
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [8]:
# Define a function to compute the mean distance between a sample and all other points in the same cluster
def intra_cluster_distance(sample, cluster):
    distances = [cosine_similarity(sample, other) for other in cluster]
    return sum(distances) / len(distances)

In [9]:
# Define a function to compute the mean distance between a sample and all other points in the next nearest cluster
def nearest_cluster_distance(sample, clusters):
    distances = []
    for other_cluster in clusters:
        if other_cluster == []:
            continue
        intra_distance = intra_cluster_distance(sample, other_cluster)
        distances.append(intra_distance)
    return sum(distances) / len(distances)

In [10]:
# Define the K-means clustering algorithm
def kmeans_clustering(data, k=3, num_iterations=20):
    # Initialize the cluster means as k distinct data points
    cluster_means = data[np.random.choice(len(data), size=k, replace=False)]
    
    # Perform the iterations
    for i in range(num_iterations):
        # Assign each data point to the nearest cluster
        clusters = [[] for j in range(k)]
        for point in data:
            distances = [cosine_similarity(point, mean) for mean in cluster_means]
            nearest_mean = np.argmax(distances)
            clusters[nearest_mean].append(point)
        # Update the cluster means
        for j in range(k):
            if clusters[j] == []:
                continue
            cluster_means[j] = np.mean(clusters[j], axis=0)

    # Assign cluster labels to data points
    kmeans_labels = np.zeros(len(data), dtype=int)  # initialize labels as zeros
    for i in range(k):
        cluster_points = clusters[i]  # points in the i-th cluster
        for point in cluster_points:
            # Find the indices of the points that match the current point in the cluster
            match_indices = np.where(np.all(data == point, axis=1))[0]
            # Assign the current cluster label to the matching points
            kmeans_labels[match_indices] = i
    
    # Compute the Silhouette coefficient
    s_values = []
    for i in range(len(data)):
        a = intra_cluster_distance(data[i], clusters[np.argmax([cosine_similarity(data[i], mean) for mean in cluster_means])])
        b = nearest_cluster_distance(data[i], clusters)
        s = (b - a) / max(a, b)
        s_values.append(s)
    silhouette_coefficient = np.mean(s_values)
           
    return clusters, silhouette_coefficient, kmeans_labels

In [11]:
# Save the clustering information to a file
def save_cluster(clusters, filename):
    with open(filename, "w") as f:
        f.write(f"{k} Clusters for best k value :\n")
        for j, cluster in enumerate(clusters):
            f.write(f"Cluster {j+1} have {len(cluster)} data points:\n")
            f.write(str(sorted(sales_data.index[sales_data[sales_data.columns[-1]] == j].tolist())))
            f.write("\n\n")

In [12]:
# Find optimal value of k
s_best = -1
k_best = -1
for k in range(3, 7):
    # K-means clustering
    kmeans_clusters, s, kmeans_labels = kmeans_clustering(encoded_sales_data, k, num_iterations=20)

    if s > s_best:
        s_best = s
        k_best = k
    
        sales_data["KmeansCluster"] = kmeans_labels   # Add the KmeansCluster column
        save_cluster(kmeans_clusters, "kmeans.txt") # Save the kmeans clustering information to a file for best k value
        
    # Print Clusters Info
    print(f"Clusters for k = {k}:")
    for j, cluster in enumerate(kmeans_clusters):
        print(f"Cluster {j+1} have {len(cluster)} data points")
    print(f"Silhouette coefficient for k = {k}: {s}\n")
    
print('Optimal value of k:', k_best)
sales_data.head(10)

Clusters for k = 3:
Cluster 1 have 191 data points
Cluster 2 have 221 data points
Cluster 3 have 588 data points
Silhouette coefficient for k = 3: -0.06601669572319607

Clusters for k = 4:
Cluster 1 have 68 data points
Cluster 2 have 221 data points
Cluster 3 have 555 data points
Cluster 4 have 156 data points
Silhouette coefficient for k = 4: -0.12573718424975047

Clusters for k = 5:
Cluster 1 have 74 data points
Cluster 2 have 221 data points
Cluster 3 have 47 data points
Cluster 4 have 113 data points
Cluster 5 have 545 data points
Silhouette coefficient for k = 5: -0.14099967295886978

Clusters for k = 6:
Cluster 1 have 540 data points
Cluster 2 have 39 data points
Cluster 3 have 221 data points
Cluster 4 have 64 data points
Cluster 5 have 103 data points
Cluster 6 have 33 data points
Silhouette coefficient for k = 6: -0.16170881816277882

Optimal value of k: 3


,Country,Sales Rep,Sales,Year of Transaction Date,Category,ID,KmeansCluster
136044,Spain,Ralph Ritter,530.868,2013,ES,2239615,1
147440,United Kingdom,Frank Atkinson,295.452,2013,MX,101525,2
230105,Sudan,Shirley Schmidt,39.480,2011,CA,124464,2
170724,France,Elizabeth Moffitt,1041.336,2012,ID,29424,2
260142,United Kingdom,Patrick O'Donnell,195.440,2011,MX,106390,2
3062,United States,Rob Dowd,5.220,2014,CA,131653,2
226283,Syria,Chuck Sachs,167.724,2011,US,137309,2
250959,Spain,Gary Mitchum,758.352,2011,CA,138940,2
216035,Senegal,Erica Hernandez,633.480,2012,SG,2990,0
247379,Philippines,Neil Französisch,173.610,2011,ES,3897276,1


In [13]:
def single_linkage_clustering(X, k):
    """Perform single linkage agglomerative clustering using cosine similarity as the distance measure"""
    n_samples = X.shape[0]
    distances = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            if i == j:
                continue
            distances[i, j] = 1 - cosine_similarity(X[i], X[j])
    
    # Initialize clusters with each sample as its own cluster
    clusters = [[i] for i in range(n_samples)]
    
    # Merge clusters until the desired number of clusters is reached
    while len(clusters) > k:
        min_distance = np.inf
        merge_indices = (0, 0)
        # Find the two clusters with the smallest distance between them
        for i in range(len(clusters)):
            for j in range(i+1, len(clusters)):
                for c1 in clusters[i]:
                    for c2 in clusters[j]:
                        distance = distances[c1, c2]
                        if distance < min_distance:
                            min_distance = distance
                            merge_indices = (i, j)
        
        # Merge the two clusters with the smallest distance
        i, j = merge_indices
        clusters[i] += clusters[j]
        del clusters[j]
        
    labels = {}
    for i, cluster in enumerate(clusters):
        for point in cluster:
            labels[point] = i
    agg_labels = [labels[i] for i in range(len(labels))]
    
    return clusters, agg_labels

In [17]:
# run agglomerative clustering algorithm
agg_clusters, agg_labels = single_linkage_clustering(encoded_sales_data, k_best)

sales_data["AggCluster"] = agg_labels # Add the AggCluster column   
save_cluster(agg_clusters, "agglomerative.txt") # Save the agglomerative clustering information to a file for best k value

sales_data.head(10)

,Country,Sales Rep,Sales,Year of Transaction Date,Category,ID,KmeansCluster,AggCluster
136044,Spain,Ralph Ritter,530.868,2013,ES,2239615,1,0
147440,United Kingdom,Frank Atkinson,295.452,2013,MX,101525,2,0
230105,Sudan,Shirley Schmidt,39.480,2011,CA,124464,2,0
170724,France,Elizabeth Moffitt,1041.336,2012,ID,29424,2,0
260142,United Kingdom,Patrick O'Donnell,195.440,2011,MX,106390,2,0
3062,United States,Rob Dowd,5.220,2014,CA,131653,2,0
226283,Syria,Chuck Sachs,167.724,2011,US,137309,2,0
250959,Spain,Gary Mitchum,758.352,2011,CA,138940,2,0
216035,Senegal,Erica Hernandez,633.480,2012,SG,2990,0,0
247379,Philippines,Neil Französisch,173.610,2011,ES,3897276,1,0


In [18]:
k = k_best
kmeans_labels = sales_data['KmeansCluster'].tolist()
kmeans_labels = np.asarray(kmeans_labels, dtype=int)
agg_labels = np.asarray(agg_labels, dtype=int)

# compute Jaccard similarity
jaccard_similarities = np.zeros((k, k))
for i in range(k):
    for j in range(k):
        intersection = len(kmeans_labels[(agg_labels == j) & (kmeans_labels == i)])
        union = len(kmeans_labels[(agg_labels == j) | (kmeans_labels == i)])
        jaccard_similarities[i, j] = intersection / union
print(jaccard_similarities)

[[0.189      0.0052356  0.0052356 ]
 [0.22144289 0.         0.        ]
 [0.58917836 0.         0.        ]]
